In [ ]:
conda install -c plotly plotly=4.14.3

In [1]:
import pandas as pd
import sqlite3 as sql
import plotly.express as px
import plotly.graph_objects as go

In [2]:
connection = sql.connect("./SQL_NoSQL_Survey.db")

In [3]:
#These queries return the total number of distinct repositories of the survey, for the 5 input datasets
countNumberOfRepositories = '''SELECT COUNT(DISTINCT REPOSITORY_NAME) AS TOTAL_NUMBER_OF_REPOSITORIES FROM SQL_NOSQL_REPOSITORY_DEPENDENCIES;'''
countNumberOfFilteredRepositories = '''SELECT COUNT(DISTINCT REPOSITORY_NAME) AS TOTAL_NUMBER_OF_FILTERED_REPOSITORIES FROM SQL_NOSQL_FILTERED_REPOSITORY_DEPENDENCIES;'''

#This query returns the total number of filtered repositories of the survey per dataset version
countNumberOfFilteredRepositoriesByDatasetVersion = '''SELECT DATASET_VERSION, COUNT(DISTINCT REPOSITORY_NAME) AS NUMBER_OF_FILTERED_REPOSITORIES FROM SQL_NOSQL_FILTERED_REPOSITORY_DEPENDENCIES GROUP BY DATASET_VERSION;'''

dfCountNumberOfRepositories = pd.read_sql_query(countNumberOfRepositories, connection)
dfCountNumberOfFilteredRepositories = pd.read_sql_query(countNumberOfFilteredRepositories, connection)
dfCountNumberOfFilteredRepositoriesByDatasetVersion = pd.read_sql_query(countNumberOfFilteredRepositoriesByDatasetVersion, connection)

print(dfCountNumberOfRepositories)
print(dfCountNumberOfFilteredRepositories)
print(dfCountNumberOfFilteredRepositoriesByDatasetVersion)

   TOTAL_NUMBER_OF_REPOSITORIES
0                       1395581
   TOTAL_NUMBER_OF_FILTERED_REPOSITORIES
0                                  42176
   DATASET_VERSION  NUMBER_OF_FILTERED_REPOSITORIES
0         20170721                            28249
1         20171129                            30252
2         20180313                            31133
3         20181222                            33754
4         20200112                            39776


In [21]:
#This query returns the total number of filtered repositories of the survey per dataset version and data model
countNumberOfRepositories = '''SELECT DATASET_VERSION, DBMS_TYPE, COUNT(DISTINCT REPOSITORY_NAME) AS NUMBER_OF_FILTERED_REPOSITORIES FROM SQL_NOSQL_FILTERED_REPOSITORY_DEPENDENCIES GROUP BY DBMS_TYPE, DATASET_VERSION;'''

dfCountNumberOfRepositories = pd.read_sql_query(countNumberOfRepositories, connection)

print(dfCountNumberOfRepositories)

    DATASET_VERSION     DBMS_TYPE  NUMBER_OF_FILTERED_REPOSITORIES
0          20170721      ColumnDB                              485
1          20171129      ColumnDB                              537
2          20180313      ColumnDB                              539
3          20181222      ColumnDB                              727
4          20200112      ColumnDB                             1147
5          20170721    DocumentDB                             6980
6          20171129    DocumentDB                             7541
7          20180313    DocumentDB                             7745
8          20181222    DocumentDB                             8459
9          20200112    DocumentDB                            10261
10         20170721       GraphDB                              301
11         20171129       GraphDB                              330
12         20180313       GraphDB                              336
13         20181222       GraphDB                             

In [23]:
#This query returns the total number of filtered repositories of the survey per database implementation in 2020
countNumberOfRepositories = '''SELECT DBMS, COUNT(DISTINCT REPOSITORY_NAME) AS NUMBER_OF_FILTERED_REPOSITORIES FROM SQL_NOSQL_FILTERED_REPOSITORY_DEPENDENCIES WHERE DATASET_VERSION = '20200112' GROUP BY DBMS ORDER BY COUNT(DISTINCT REPOSITORY_NAME) DESC;'''

dfCountNumberOfRepositories = pd.read_sql_query(countNumberOfRepositories, connection)

print(dfCountNumberOfRepositories)

         DBMS  NUMBER_OF_FILTERED_REPOSITORIES
0  PostgreSQL                            12959
1     MongoDB                            10154
2       MySQL                             8962
3      SQLite                             8729
4       Redis                             7943
5   Memcached                             1993
6   Cassandra                             1018
7       Neo4j                              418
8       HBase                              161
9   Couchbase                              129


In [24]:
#This query returns the total number of filtered repositories of the survey per data model and dependency programming language in 2020
countNumberOfRepositories = '''SELECT DBMS_TYPE, DEPENDENCY_LANGUAGE, COUNT(DISTINCT REPOSITORY_NAME) AS NUMBER_OF_FILTERED_REPOSITORIES FROM SQL_NOSQL_FILTERED_REPOSITORY_DEPENDENCIES WHERE DATASET_VERSION = '20200112' GROUP BY DBMS_TYPE, DEPENDENCY_LANGUAGE;'''

dfCountNumberOfRepositories = pd.read_sql_query(countNumberOfRepositories, connection)

print(dfCountNumberOfRepositories)

       DBMS_TYPE    DEPENDENCY_LANGUAGE  NUMBER_OF_FILTERED_REPOSITORIES
0       ColumnDB  C#, C++, F#, VB, .NET                               27
1       ColumnDB                     Go                               60
2       ColumnDB                   Java                              394
3       ColumnDB             JavaScript                               80
4       ColumnDB                    PHP                                4
5       ColumnDB                 Python                              551
6       ColumnDB                   Ruby                               36
7     DocumentDB  C#, C++, F#, VB, .NET                              286
8     DocumentDB                Clojure                               40
9     DocumentDB                     Go                              141
10    DocumentDB                   Java                              803
11    DocumentDB             JavaScript                             6297
12    DocumentDB                    PHP            

In [26]:
#This query returns the total number of filtered repositories of the survey per dependency in 2020 and dependency programming language in 2020
countNumberOfRepositories = '''SELECT DEPENDENCY_NAME, DEPENDENCY_LANGUAGE, ODM AS 'ORM/ONM', COUNT(DISTINCT REPOSITORY_NAME) AS NUMBER_OF_FILTERED_REPOSITORIES FROM SQL_NOSQL_FILTERED_REPOSITORY_DEPENDENCIES WHERE DATASET_VERSION = '20200112' GROUP BY DEPENDENCY_NAME, DEPENDENCY_LANGUAGE ORDER BY COUNT(DISTINCT REPOSITORY_NAME) DESC;'''

dfCountNumberOfRepositories = pd.read_sql_query(countNumberOfRepositories, connection)

print(dfCountNumberOfRepositories)

         DEPENDENCY_NAME DEPENDENCY_LANGUAGE  ODM  \
0                     pg                Ruby    0   
1                sqlite3                Ruby    0   
2               mongoose          JavaScript    1   
3               psycopg2              Python    0   
4                mongodb          JavaScript    0   
..                   ...                 ...  ...   
308             starbase              Python    0   
309             tornadis              Python    0   
310            toshihiko          JavaScript    1   
311          ts-postgres          JavaScript    0   
312  vinelab/neoeloquent                 PHP    1   

     NUMBER_OF_FILTERED_REPOSITORIES  
0                               6125  
1                               5798  
2                               4424  
3                               3683  
4                               3406  
..                               ...  
308                                1  
309                                1  
310         

In [27]:
#This query returns the total number of filtered repositories of the survey per combination of data model in 2020 and dependency programming language in 2020
countNumberOfRepositories = '''SELECT DATASET_VERSION, DATA_MODELS, NUMBER_OF_FILTERED_REPOSITORIES FROM (
    --Only relational DB
    SELECT DATASET_VERSION, 'R' AS 'DATA_MODELS', COUNT(DISTINCT REPOSITORY_NAME) AS NUMBER_OF_FILTERED_REPOSITORIES FROM SQL_NOSQL_FILTERED_REPOSITORY_DEPENDENCIES WHERE REPOSITORY_NAME IN (
        SELECT REPOSITORY_NAME FROM SQL_NOSQL_REPOSITORY_WITH_DBMS_TYPE WHERE HAS_RELATIONAL_DB_DEPENDENCY = true AND HAS_DOCUMENT_DB_DEPENDENCY = false AND HAS_KEY_VALUE_DB_DEPENDENCY = false AND HAS_COLUMN_DB_DEPENDENCY = false AND HAS_GRAPH_DB_DEPENDENCY = false
    ) GROUP BY DATASET_VERSION
    UNION
    --Only document DB
    SELECT DATASET_VERSION, 'D' AS 'DATA_MODELS', COUNT(DISTINCT REPOSITORY_NAME) AS NUMBER_OF_FILTERED_REPOSITORIES FROM SQL_NOSQL_FILTERED_REPOSITORY_DEPENDENCIES WHERE REPOSITORY_NAME IN (
        SELECT REPOSITORY_NAME FROM SQL_NOSQL_REPOSITORY_WITH_DBMS_TYPE WHERE HAS_RELATIONAL_DB_DEPENDENCY = false AND HAS_DOCUMENT_DB_DEPENDENCY = true AND HAS_KEY_VALUE_DB_DEPENDENCY = false AND HAS_COLUMN_DB_DEPENDENCY = false AND HAS_GRAPH_DB_DEPENDENCY = false
    ) GROUP BY DATASET_VERSION
    UNION
    --Only key-value DB
    SELECT DATASET_VERSION, 'K' AS 'DATA_MODELS', COUNT(DISTINCT REPOSITORY_NAME) AS NUMBER_OF_FILTERED_REPOSITORIES FROM SQL_NOSQL_FILTERED_REPOSITORY_DEPENDENCIES WHERE REPOSITORY_NAME IN (
        SELECT REPOSITORY_NAME FROM SQL_NOSQL_REPOSITORY_WITH_DBMS_TYPE WHERE HAS_RELATIONAL_DB_DEPENDENCY = false AND HAS_DOCUMENT_DB_DEPENDENCY = false AND HAS_KEY_VALUE_DB_DEPENDENCY = true AND HAS_COLUMN_DB_DEPENDENCY = false AND HAS_GRAPH_DB_DEPENDENCY = false
    ) GROUP BY DATASET_VERSION
    UNION
    --Only column DB
    SELECT DATASET_VERSION, 'C' AS 'DATA_MODELS', COUNT(DISTINCT REPOSITORY_NAME) AS NUMBER_OF_FILTERED_REPOSITORIES FROM SQL_NOSQL_FILTERED_REPOSITORY_DEPENDENCIES WHERE REPOSITORY_NAME IN (
        SELECT REPOSITORY_NAME FROM SQL_NOSQL_REPOSITORY_WITH_DBMS_TYPE WHERE HAS_RELATIONAL_DB_DEPENDENCY = false AND HAS_DOCUMENT_DB_DEPENDENCY = false AND HAS_KEY_VALUE_DB_DEPENDENCY = false AND HAS_COLUMN_DB_DEPENDENCY = true AND HAS_GRAPH_DB_DEPENDENCY = false
    ) GROUP BY DATASET_VERSION
    UNION
    --Only graph DB
    SELECT DATASET_VERSION, 'G' AS 'DATA_MODELS', COUNT(DISTINCT REPOSITORY_NAME) AS NUMBER_OF_FILTERED_REPOSITORIES FROM SQL_NOSQL_FILTERED_REPOSITORY_DEPENDENCIES WHERE REPOSITORY_NAME IN (
        SELECT REPOSITORY_NAME FROM SQL_NOSQL_REPOSITORY_WITH_DBMS_TYPE WHERE HAS_RELATIONAL_DB_DEPENDENCY = false AND HAS_DOCUMENT_DB_DEPENDENCY = false AND HAS_KEY_VALUE_DB_DEPENDENCY = false AND HAS_COLUMN_DB_DEPENDENCY = false AND HAS_GRAPH_DB_DEPENDENCY = true
    ) GROUP BY DATASET_VERSION
    UNION
    --Number of projects having two database types
    --Relational DB & Key-Value DB
    SELECT DATASET_VERSION, 'R + K' AS 'DATA_MODELS', COUNT(DISTINCT REPOSITORY_NAME) AS NUMBER_OF_FILTERED_REPOSITORIES FROM SQL_NOSQL_FILTERED_REPOSITORY_DEPENDENCIES WHERE REPOSITORY_NAME IN (
        SELECT REPOSITORY_NAME FROM SQL_NOSQL_REPOSITORY_WITH_DBMS_TYPE WHERE HAS_RELATIONAL_DB_DEPENDENCY = true AND HAS_DOCUMENT_DB_DEPENDENCY = false AND HAS_KEY_VALUE_DB_DEPENDENCY = true AND HAS_COLUMN_DB_DEPENDENCY = false AND HAS_GRAPH_DB_DEPENDENCY = false
    ) GROUP BY DATASET_VERSION
    UNION
    --Relational DB & Document DB
    SELECT DATASET_VERSION, 'R + D' AS 'DATA_MODELS', COUNT(DISTINCT REPOSITORY_NAME) AS NUMBER_OF_FILTERED_REPOSITORIES FROM SQL_NOSQL_FILTERED_REPOSITORY_DEPENDENCIES WHERE REPOSITORY_NAME IN (
        SELECT REPOSITORY_NAME FROM SQL_NOSQL_REPOSITORY_WITH_DBMS_TYPE WHERE HAS_RELATIONAL_DB_DEPENDENCY = true AND HAS_DOCUMENT_DB_DEPENDENCY = true AND HAS_KEY_VALUE_DB_DEPENDENCY = false AND HAS_COLUMN_DB_DEPENDENCY = false AND HAS_GRAPH_DB_DEPENDENCY = false
    ) GROUP BY DATASET_VERSION
    UNION
    --Document DB & Key-Value DB
    SELECT DATASET_VERSION, 'D + K' AS 'DATA_MODELS', COUNT(DISTINCT REPOSITORY_NAME) AS NUMBER_OF_FILTERED_REPOSITORIES FROM SQL_NOSQL_FILTERED_REPOSITORY_DEPENDENCIES WHERE REPOSITORY_NAME IN (
        SELECT REPOSITORY_NAME FROM SQL_NOSQL_REPOSITORY_WITH_DBMS_TYPE WHERE HAS_RELATIONAL_DB_DEPENDENCY = false AND HAS_DOCUMENT_DB_DEPENDENCY = true AND HAS_KEY_VALUE_DB_DEPENDENCY = true AND HAS_COLUMN_DB_DEPENDENCY = false AND HAS_GRAPH_DB_DEPENDENCY = false
    ) GROUP BY DATASET_VERSION
    UNION
    --Relational DB & Column DB
    SELECT DATASET_VERSION, 'R + C' AS 'DATA_MODELS', COUNT(DISTINCT REPOSITORY_NAME) AS NUMBER_OF_FILTERED_REPOSITORIES FROM SQL_NOSQL_FILTERED_REPOSITORY_DEPENDENCIES WHERE REPOSITORY_NAME IN (
        SELECT REPOSITORY_NAME FROM SQL_NOSQL_REPOSITORY_WITH_DBMS_TYPE WHERE HAS_RELATIONAL_DB_DEPENDENCY = true AND HAS_DOCUMENT_DB_DEPENDENCY = false AND HAS_KEY_VALUE_DB_DEPENDENCY = false AND HAS_COLUMN_DB_DEPENDENCY = true AND HAS_GRAPH_DB_DEPENDENCY = false
    ) GROUP BY DATASET_VERSION
    UNION
    --Key-Value DB & Column DB
    SELECT DATASET_VERSION, 'K + C' AS 'DATA_MODELS', COUNT(DISTINCT REPOSITORY_NAME) AS NUMBER_OF_FILTERED_REPOSITORIES FROM SQL_NOSQL_FILTERED_REPOSITORY_DEPENDENCIES WHERE REPOSITORY_NAME IN (
        SELECT REPOSITORY_NAME FROM SQL_NOSQL_REPOSITORY_WITH_DBMS_TYPE WHERE HAS_RELATIONAL_DB_DEPENDENCY = false AND HAS_DOCUMENT_DB_DEPENDENCY = false AND HAS_KEY_VALUE_DB_DEPENDENCY = true AND HAS_COLUMN_DB_DEPENDENCY = true AND HAS_GRAPH_DB_DEPENDENCY = false
    ) GROUP BY DATASET_VERSION
    UNION
    --Document DB & Column DB
    SELECT DATASET_VERSION, 'D + C' AS 'DATA_MODELS', COUNT(DISTINCT REPOSITORY_NAME) AS NUMBER_OF_FILTERED_REPOSITORIES FROM SQL_NOSQL_FILTERED_REPOSITORY_DEPENDENCIES WHERE REPOSITORY_NAME IN (
        SELECT REPOSITORY_NAME FROM SQL_NOSQL_REPOSITORY_WITH_DBMS_TYPE WHERE HAS_RELATIONAL_DB_DEPENDENCY = false AND HAS_DOCUMENT_DB_DEPENDENCY = true AND HAS_KEY_VALUE_DB_DEPENDENCY = false AND HAS_COLUMN_DB_DEPENDENCY = true AND HAS_GRAPH_DB_DEPENDENCY = false
    ) GROUP BY DATASET_VERSION
    UNION
    --Relational DB & Graph DB
    SELECT DATASET_VERSION, 'R + G' AS 'DATA_MODELS', COUNT(DISTINCT REPOSITORY_NAME) AS NUMBER_OF_FILTERED_REPOSITORIES FROM SQL_NOSQL_FILTERED_REPOSITORY_DEPENDENCIES WHERE REPOSITORY_NAME IN (
        SELECT REPOSITORY_NAME FROM SQL_NOSQL_REPOSITORY_WITH_DBMS_TYPE WHERE HAS_RELATIONAL_DB_DEPENDENCY = true AND HAS_DOCUMENT_DB_DEPENDENCY = false AND HAS_KEY_VALUE_DB_DEPENDENCY = false AND HAS_COLUMN_DB_DEPENDENCY = false AND HAS_GRAPH_DB_DEPENDENCY = true
    ) GROUP BY DATASET_VERSION
    UNION
    --Document DB & Graph DB
    SELECT DATASET_VERSION, 'D + G' AS 'DATA_MODELS', COUNT(DISTINCT REPOSITORY_NAME) AS NUMBER_OF_FILTERED_REPOSITORIES FROM SQL_NOSQL_FILTERED_REPOSITORY_DEPENDENCIES WHERE REPOSITORY_NAME IN (
        SELECT REPOSITORY_NAME FROM SQL_NOSQL_REPOSITORY_WITH_DBMS_TYPE WHERE HAS_RELATIONAL_DB_DEPENDENCY = false AND HAS_DOCUMENT_DB_DEPENDENCY = true AND HAS_KEY_VALUE_DB_DEPENDENCY = false AND HAS_COLUMN_DB_DEPENDENCY = false AND HAS_GRAPH_DB_DEPENDENCY = true
    ) GROUP BY DATASET_VERSION
    UNION
    --Key-Value & Graph DB
    SELECT DATASET_VERSION, 'K + G' AS 'DATA_MODELS', COUNT(DISTINCT REPOSITORY_NAME) AS NUMBER_OF_FILTERED_REPOSITORIES FROM SQL_NOSQL_FILTERED_REPOSITORY_DEPENDENCIES WHERE REPOSITORY_NAME IN (
        SELECT REPOSITORY_NAME FROM SQL_NOSQL_REPOSITORY_WITH_DBMS_TYPE WHERE HAS_RELATIONAL_DB_DEPENDENCY = false AND HAS_DOCUMENT_DB_DEPENDENCY = false AND HAS_KEY_VALUE_DB_DEPENDENCY = true AND HAS_COLUMN_DB_DEPENDENCY = false AND HAS_GRAPH_DB_DEPENDENCY = true
    ) GROUP BY DATASET_VERSION
    UNION
    --Column DB & Graph DB
    SELECT DATASET_VERSION, 'C + G' AS 'DATA_MODELS', COUNT(DISTINCT REPOSITORY_NAME) AS NUMBER_OF_FILTERED_REPOSITORIES FROM SQL_NOSQL_FILTERED_REPOSITORY_DEPENDENCIES WHERE REPOSITORY_NAME IN (
        SELECT REPOSITORY_NAME FROM SQL_NOSQL_REPOSITORY_WITH_DBMS_TYPE WHERE HAS_RELATIONAL_DB_DEPENDENCY = false AND HAS_DOCUMENT_DB_DEPENDENCY = false AND HAS_KEY_VALUE_DB_DEPENDENCY = false AND HAS_COLUMN_DB_DEPENDENCY = true AND HAS_GRAPH_DB_DEPENDENCY = true
    ) GROUP BY DATASET_VERSION
    UNION
    --Number of projects having three database types
    --Relational DB & Document DB & Key-Value DB
    SELECT DATASET_VERSION, 'R + D + K' AS 'DATA_MODELS', COUNT(DISTINCT REPOSITORY_NAME) AS NUMBER_OF_FILTERED_REPOSITORIES FROM SQL_NOSQL_FILTERED_REPOSITORY_DEPENDENCIES WHERE REPOSITORY_NAME IN (
        SELECT REPOSITORY_NAME FROM SQL_NOSQL_REPOSITORY_WITH_DBMS_TYPE WHERE HAS_RELATIONAL_DB_DEPENDENCY = true AND HAS_DOCUMENT_DB_DEPENDENCY = true AND HAS_KEY_VALUE_DB_DEPENDENCY = true AND HAS_COLUMN_DB_DEPENDENCY = false AND HAS_GRAPH_DB_DEPENDENCY = false
    ) GROUP BY DATASET_VERSION
    UNION
    --Relational DB & Key-Value DB & Column DB
    SELECT DATASET_VERSION, 'R + K + C' AS 'DATA_MODELS', COUNT(DISTINCT REPOSITORY_NAME) AS NUMBER_OF_FILTERED_REPOSITORIES FROM SQL_NOSQL_FILTERED_REPOSITORY_DEPENDENCIES WHERE REPOSITORY_NAME IN (
        SELECT REPOSITORY_NAME FROM SQL_NOSQL_REPOSITORY_WITH_DBMS_TYPE WHERE HAS_RELATIONAL_DB_DEPENDENCY = true AND HAS_DOCUMENT_DB_DEPENDENCY = false AND HAS_KEY_VALUE_DB_DEPENDENCY = true AND HAS_COLUMN_DB_DEPENDENCY = true AND HAS_GRAPH_DB_DEPENDENCY = false
    ) GROUP BY DATASET_VERSION
    UNION
    --Relational DB & Document DB & Column DB
    SELECT DATASET_VERSION, 'R + D + C' AS 'DATA_MODELS', COUNT(DISTINCT REPOSITORY_NAME) AS NUMBER_OF_FILTERED_REPOSITORIES FROM SQL_NOSQL_FILTERED_REPOSITORY_DEPENDENCIES WHERE REPOSITORY_NAME IN (
        SELECT REPOSITORY_NAME FROM SQL_NOSQL_REPOSITORY_WITH_DBMS_TYPE WHERE HAS_RELATIONAL_DB_DEPENDENCY = true AND HAS_DOCUMENT_DB_DEPENDENCY = true AND HAS_KEY_VALUE_DB_DEPENDENCY = false AND HAS_COLUMN_DB_DEPENDENCY = true AND HAS_GRAPH_DB_DEPENDENCY = false
    ) GROUP BY DATASET_VERSION
    UNION
    --Document DB & Key-Value DB & Column DB
    SELECT DATASET_VERSION, 'D + K + C' AS 'DATA_MODELS', COUNT(DISTINCT REPOSITORY_NAME) AS NUMBER_OF_FILTERED_REPOSITORIES FROM SQL_NOSQL_FILTERED_REPOSITORY_DEPENDENCIES WHERE REPOSITORY_NAME IN (
        SELECT REPOSITORY_NAME FROM SQL_NOSQL_REPOSITORY_WITH_DBMS_TYPE WHERE HAS_RELATIONAL_DB_DEPENDENCY = false AND HAS_DOCUMENT_DB_DEPENDENCY = true AND HAS_KEY_VALUE_DB_DEPENDENCY = true AND HAS_COLUMN_DB_DEPENDENCY = true AND HAS_GRAPH_DB_DEPENDENCY = false
    ) GROUP BY DATASET_VERSION
    UNION
    --Relational DB & Document DB & Graph DB
    SELECT DATASET_VERSION, 'R + D + G' AS 'DATA_MODELS', COUNT(DISTINCT REPOSITORY_NAME) AS NUMBER_OF_FILTERED_REPOSITORIES FROM SQL_NOSQL_FILTERED_REPOSITORY_DEPENDENCIES WHERE REPOSITORY_NAME IN (
        SELECT REPOSITORY_NAME FROM SQL_NOSQL_REPOSITORY_WITH_DBMS_TYPE WHERE HAS_RELATIONAL_DB_DEPENDENCY = true AND HAS_DOCUMENT_DB_DEPENDENCY = true AND HAS_KEY_VALUE_DB_DEPENDENCY = false AND HAS_COLUMN_DB_DEPENDENCY = false AND HAS_GRAPH_DB_DEPENDENCY = true
    ) GROUP BY DATASET_VERSION
    UNION
    --Document DB & Key-Value DB & Graph DB
    SELECT DATASET_VERSION, 'D + K + G' AS 'DATA_MODELS', COUNT(DISTINCT REPOSITORY_NAME) AS NUMBER_OF_FILTERED_REPOSITORIES FROM SQL_NOSQL_FILTERED_REPOSITORY_DEPENDENCIES WHERE REPOSITORY_NAME IN (
        SELECT REPOSITORY_NAME FROM SQL_NOSQL_REPOSITORY_WITH_DBMS_TYPE WHERE HAS_RELATIONAL_DB_DEPENDENCY = false AND HAS_DOCUMENT_DB_DEPENDENCY = true AND HAS_KEY_VALUE_DB_DEPENDENCY = true AND HAS_COLUMN_DB_DEPENDENCY = false AND HAS_GRAPH_DB_DEPENDENCY = true
    ) GROUP BY DATASET_VERSION
    UNION
    --Relational DB & Key-Value DB & Graph DB
    SELECT DATASET_VERSION, 'R + K + G' AS 'DATA_MODELS', COUNT(DISTINCT REPOSITORY_NAME) AS NUMBER_OF_FILTERED_REPOSITORIES FROM SQL_NOSQL_FILTERED_REPOSITORY_DEPENDENCIES WHERE REPOSITORY_NAME IN (
        SELECT REPOSITORY_NAME FROM SQL_NOSQL_REPOSITORY_WITH_DBMS_TYPE WHERE HAS_RELATIONAL_DB_DEPENDENCY = true AND HAS_DOCUMENT_DB_DEPENDENCY = false AND HAS_KEY_VALUE_DB_DEPENDENCY = true AND HAS_COLUMN_DB_DEPENDENCY = false AND HAS_GRAPH_DB_DEPENDENCY = true
    ) GROUP BY DATASET_VERSION
    UNION
    --Relational DB & Column DB & Graph DB
    SELECT DATASET_VERSION, 'R + C + G' AS 'DATA_MODELS', COUNT(DISTINCT REPOSITORY_NAME) AS NUMBER_OF_FILTERED_REPOSITORIES FROM SQL_NOSQL_FILTERED_REPOSITORY_DEPENDENCIES WHERE REPOSITORY_NAME IN (
        SELECT REPOSITORY_NAME FROM SQL_NOSQL_REPOSITORY_WITH_DBMS_TYPE WHERE HAS_RELATIONAL_DB_DEPENDENCY = true AND HAS_DOCUMENT_DB_DEPENDENCY = false AND HAS_KEY_VALUE_DB_DEPENDENCY = false AND HAS_COLUMN_DB_DEPENDENCY = true AND HAS_GRAPH_DB_DEPENDENCY = true
    ) GROUP BY DATASET_VERSION
    UNION
    --Document DB & Column DB & Graph DB
    SELECT DATASET_VERSION, 'D + C + G' AS 'DATA_MODELS', COUNT(DISTINCT REPOSITORY_NAME) AS NUMBER_OF_FILTERED_REPOSITORIES FROM SQL_NOSQL_FILTERED_REPOSITORY_DEPENDENCIES WHERE REPOSITORY_NAME IN (
        SELECT REPOSITORY_NAME FROM SQL_NOSQL_REPOSITORY_WITH_DBMS_TYPE WHERE HAS_RELATIONAL_DB_DEPENDENCY = false AND HAS_DOCUMENT_DB_DEPENDENCY = true AND HAS_KEY_VALUE_DB_DEPENDENCY = false AND HAS_COLUMN_DB_DEPENDENCY = true AND HAS_GRAPH_DB_DEPENDENCY = true
    ) GROUP BY DATASET_VERSION
    UNION
    --Key-Value DB & Column DB & Graph DB
    SELECT DATASET_VERSION, 'K + C + G' AS 'DATA_MODELS', COUNT(DISTINCT REPOSITORY_NAME) AS NUMBER_OF_FILTERED_REPOSITORIES FROM SQL_NOSQL_FILTERED_REPOSITORY_DEPENDENCIES WHERE REPOSITORY_NAME IN (
        SELECT REPOSITORY_NAME FROM SQL_NOSQL_REPOSITORY_WITH_DBMS_TYPE WHERE HAS_RELATIONAL_DB_DEPENDENCY = false AND HAS_DOCUMENT_DB_DEPENDENCY = false AND HAS_KEY_VALUE_DB_DEPENDENCY = true AND HAS_COLUMN_DB_DEPENDENCY = true AND HAS_GRAPH_DB_DEPENDENCY = true
    ) GROUP BY DATASET_VERSION
    UNION
    --Number of projects having four database types
    --Relational DB & Document DB & Key-Value DB & Column DB
    SELECT DATASET_VERSION, 'R + D + K + C' AS 'DATA_MODELS', COUNT(DISTINCT REPOSITORY_NAME) AS NUMBER_OF_FILTERED_REPOSITORIES FROM SQL_NOSQL_FILTERED_REPOSITORY_DEPENDENCIES WHERE REPOSITORY_NAME IN (
        SELECT REPOSITORY_NAME FROM SQL_NOSQL_REPOSITORY_WITH_DBMS_TYPE WHERE HAS_RELATIONAL_DB_DEPENDENCY = true AND HAS_DOCUMENT_DB_DEPENDENCY = true AND HAS_KEY_VALUE_DB_DEPENDENCY = true AND HAS_COLUMN_DB_DEPENDENCY = true AND HAS_GRAPH_DB_DEPENDENCY = false
    ) GROUP BY DATASET_VERSION
    UNION
    --Relational DB & Document DB & Key-Value DB & Graph DB
    SELECT DATASET_VERSION, 'R + D + K + G' AS 'DATA_MODELS', COUNT(DISTINCT REPOSITORY_NAME) AS NUMBER_OF_FILTERED_REPOSITORIES FROM SQL_NOSQL_FILTERED_REPOSITORY_DEPENDENCIES WHERE REPOSITORY_NAME IN (
        SELECT REPOSITORY_NAME FROM SQL_NOSQL_REPOSITORY_WITH_DBMS_TYPE WHERE HAS_RELATIONAL_DB_DEPENDENCY = true AND HAS_DOCUMENT_DB_DEPENDENCY = true AND HAS_KEY_VALUE_DB_DEPENDENCY = true AND HAS_COLUMN_DB_DEPENDENCY = false AND HAS_GRAPH_DB_DEPENDENCY = true
    ) GROUP BY DATASET_VERSION
    UNION
    --Document DB & Key-Value DB & Column DB & Graph DB
    SELECT DATASET_VERSION, 'D + K + C + G' AS 'DATA_MODELS', COUNT(DISTINCT REPOSITORY_NAME) AS NUMBER_OF_FILTERED_REPOSITORIES FROM SQL_NOSQL_FILTERED_REPOSITORY_DEPENDENCIES WHERE REPOSITORY_NAME IN (
        SELECT REPOSITORY_NAME FROM SQL_NOSQL_REPOSITORY_WITH_DBMS_TYPE WHERE HAS_RELATIONAL_DB_DEPENDENCY = false AND HAS_DOCUMENT_DB_DEPENDENCY = true AND HAS_KEY_VALUE_DB_DEPENDENCY = true AND HAS_COLUMN_DB_DEPENDENCY = true AND HAS_GRAPH_DB_DEPENDENCY = true
    ) GROUP BY DATASET_VERSION
    UNION
    --Relational DB & Document DB & Column & Graph DB
    SELECT DATASET_VERSION, 'R + D + C + G' AS 'DATA_MODELS', COUNT(DISTINCT REPOSITORY_NAME) AS NUMBER_OF_FILTERED_REPOSITORIES FROM SQL_NOSQL_FILTERED_REPOSITORY_DEPENDENCIES WHERE REPOSITORY_NAME IN (
        SELECT REPOSITORY_NAME FROM SQL_NOSQL_REPOSITORY_WITH_DBMS_TYPE WHERE HAS_RELATIONAL_DB_DEPENDENCY = true AND HAS_DOCUMENT_DB_DEPENDENCY = true AND HAS_KEY_VALUE_DB_DEPENDENCY = false AND HAS_COLUMN_DB_DEPENDENCY = true AND HAS_GRAPH_DB_DEPENDENCY = true
    ) GROUP BY DATASET_VERSION
    UNION
    --Relational DB & Key-Value DB & Column & Graph DB
    SELECT DATASET_VERSION, 'R + K + C + G' AS 'DATA_MODELS', COUNT(DISTINCT REPOSITORY_NAME) AS NUMBER_OF_FILTERED_REPOSITORIES FROM SQL_NOSQL_FILTERED_REPOSITORY_DEPENDENCIES WHERE REPOSITORY_NAME IN (
        SELECT REPOSITORY_NAME FROM SQL_NOSQL_REPOSITORY_WITH_DBMS_TYPE WHERE HAS_RELATIONAL_DB_DEPENDENCY = true AND HAS_DOCUMENT_DB_DEPENDENCY = false AND HAS_KEY_VALUE_DB_DEPENDENCY = true AND HAS_COLUMN_DB_DEPENDENCY = true AND HAS_GRAPH_DB_DEPENDENCY = true
    ) GROUP BY DATASET_VERSION
    UNION
    --Number of projects having five database types
    --Relational DB & Document DB & Key-Value DB & Column DB & Graph DB
    SELECT DATASET_VERSION, 'R + D + K + C + G' AS 'DATA_MODELS', COUNT(DISTINCT REPOSITORY_NAME) AS NUMBER_OF_FILTERED_REPOSITORIES FROM SQL_NOSQL_FILTERED_REPOSITORY_DEPENDENCIES WHERE REPOSITORY_NAME IN (
        SELECT REPOSITORY_NAME FROM SQL_NOSQL_REPOSITORY_WITH_DBMS_TYPE WHERE HAS_RELATIONAL_DB_DEPENDENCY = true AND HAS_DOCUMENT_DB_DEPENDENCY = true AND HAS_KEY_VALUE_DB_DEPENDENCY = true AND HAS_COLUMN_DB_DEPENDENCY = true AND HAS_GRAPH_DB_DEPENDENCY = true
    )
)
ORDER BY DATASET_VERSION, NUMBER_OF_FILTERED_REPOSITORIES DESC;'''

dfCountNumberOfRepositories = pd.read_sql_query(countNumberOfRepositories, connection)

print(dfCountNumberOfRepositories)

     DATASET_VERSION    DATA_MODELS  NUMBER_OF_FILTERED_REPOSITORIES
0           20170721              R                            14981
1           20170721              D                             5493
2           20170721              K                             2772
3           20170721          R + K                             2753
4           20170721          D + K                              587
..               ...            ...                              ...
128         20200112          C + G                                5
129         20200112      R + K + G                                5
130         20200112      R + C + G                                2
131         20200112  R + D + C + G                                2
132         20200112      K + C + G                                1

[133 rows x 3 columns]


In [18]:
#This query returns the total number of filtered repositories of the survey which evolved their data model(s)
countNumberOfRepositories = '''SELECT COUNT(DISTINCT REPOSITORY_NAME) AS TOTAL_NUMBER_OF_FILTERED_EVOLVING_REPOSITORIES FROM SQL_NOSQL_DBMS_TYPE_SURVIVAL_ANALYSIS_REPO_GLOBAL;'''

dfCountNumberOfRepositories = pd.read_sql_query(countNumberOfRepositories, connection)

print(dfCountNumberOfRepositories)

query = '''SELECT SOURCE_STATE_TEXT AS INITIAL_VERSION,
       FINAL_STATE_TEXT AS LATEST_VERSION,
       GLOBAL_EVOLUTION_TYPE_COUNT_FILTERED AS NUMBER_OF_EVOLVING_REPOSITORIES
FROM SQL_NOSQL_DBMS_TYPE_SURVIVAL_ANALYSIS_REPO_GLOBAL
ORDER BY GLOBAL_EVOLUTION_TYPE_COUNT_FILTERED DESC;'''

survival_df = pd.read_sql_query(query, connection)

# Create dimensions
source_state_dimension = go.parcats.Dimension(values=survival_df.INITIAL_VERSION, label="First version")
final_state_dimension = go.parcats.Dimension(values=survival_df.LATEST_VERSION, label="Latest version")

# Create colors
colorscale1 = [[0, 'lightblue'], [1, 'blue']]
colorscale2 = px.colors.sequential.Peach

# Create parcats trace
fig = go.Figure(data = [go.Parcats(dimensions=[source_state_dimension, final_state_dimension],
        #line={'colorscale': colorscale, 'cmin': 0, 'cmax': 1, 'color': color, 'shape': 'hspline'},
        #line={'color_continuous_scale':px.colors.sequential.Inferno},
        line={'color':survival_df.NUMBER_OF_EVOLVING_REPOSITORIES, 'colorscale': colorscale2, 'showscale':True, 'shape':'hspline'},
        hoveron='category', hoverinfo='count',
        labelfont={'size': 20, 'family': 'Times'},
        tickfont={'size': 10, 'family': 'Times'},
        arrangement='freeform',
        )])

fig.show()

   TOTAL_NUMBER_OF_FILTERED_EVOLVING_REPOSITORIES
0                                             473


In [26]:
#This query returns the total number of filtered repositories of the survey per category of data model evolution
countNumberOfRepositories = '''SELECT GLOBAL_EVOLUTION_TYPE, COUNT() AS NUMBER_OF_FILTERED_REPOSITORIES FROM SQL_NOSQL_DBMS_TYPE_SURVIVAL_ANALYSIS_REPO_GLOBAL GROUP BY GLOBAL_EVOLUTION_TYPE;'''

dfCountNumberOfRepositories = pd.read_sql_query(countNumberOfRepositories, connection)

print(dfCountNumberOfRepositories)

print()

#This query returns the total number of filtered repositories of the survey per category of data model evolution and hybridation level
countNumberOfRepositories = '''SELECT GLOBAL_EVOLUTION_TYPE AS CAT_OF_DATA_MODEL_EVOLUTION, COUNT() AS NUMBER_OF_REPOSITORIES, CASE WHEN NB_SOURCE_DBMS_TYPES < NB_FINAL_DBMS_TYPES THEN 'More hybrid' WHEN NB_SOURCE_DBMS_TYPES = NB_FINAL_DBMS_TYPES THEN 'Unchanged' WHEN NB_SOURCE_DBMS_TYPES > NB_FINAL_DBMS_TYPES THEN 'Less hybrid' END AS GLOBAL_EVOLUTION_TYPE
FROM SQL_NOSQL_DBMS_TYPE_SURVIVAL_ANALYSIS_REPO_GLOBAL
WHERE GLOBAL_EVOLUTION_TYPE IN ('Become hybrid', 'Stay hybrid')
GROUP BY NB_SOURCE_DBMS_TYPES, NB_FINAL_DBMS_TYPES
ORDER BY NUMBER_OF_REPOSITORIES DESC;'''

dfCountNumberOfRepositories = pd.read_sql_query(countNumberOfRepositories, connection)

print(dfCountNumberOfRepositories)

  GLOBAL_EVOLUTION_TYPE  NUMBER_OF_FILTERED_REPOSITORIES
0         Become hybrid                              294
1  Become mono-database                               89
2           Stay hybrid                               55
3    Stay mono-database                               35

   CAT_OF_DATA_MODEL_EVOLUTION  NUMBER_OF_REPOSITORIES GLOBAL_EVOLUTION_TYPE
0                Become hybrid                     275           More hybrid
1                  Stay hybrid                      23           More hybrid
2                Become hybrid                      17           More hybrid
3                  Stay hybrid                      14             Unchanged
4                  Stay hybrid                       5           More hybrid
5                  Stay hybrid                       4           Less hybrid
6                  Stay hybrid                       3           More hybrid
7                Become hybrid                       2           More hybrid
8                  St

In [10]:
connection.close()